## Usage test of the wrapper


In [1]:
# imports
import pandas as pd
import numpy as np

import logging

# from wrapper import RustDataModule
from rust_time_series.rust_time_series import (
    BaseDataSet,
    DatasetType,
    ImputeStrategy,
    SplittingStrategy,
)

In [2]:
# set up logging
logging.basicConfig(level=logging.INFO)

In [4]:
# create dummy 2D array
dummy = np.random.randint(0, 100, (100, 10)).astype(float)

# Create a RustTimeSeries instance
dummy_ts = BaseDataSet(dummy, 3, 3, 1)
dummy_ts.set_to_100()


dummy

array([[100.,  91.,  67.,  72.,  74.,  88.,  88.,  48.,  14.,  28.],
       [  1.,  82.,  10.,  56.,  40.,  63.,  85.,  50.,   1.,  67.],
       [  3.,  19.,  32.,   9.,  44.,  54.,  97.,  70.,  68.,  68.],
       [ 84.,  76.,  15.,  56.,  58.,  22.,  42.,  41.,  65.,  55.],
       [ 17.,   8.,   9.,  75.,  27.,  21.,  82.,  27.,  97.,  51.],
       [ 43.,  92.,  37.,  82.,  90.,  65.,   1.,  40.,   0.,  85.],
       [ 19.,   0.,  76.,  95.,  12.,  38.,  29.,  11.,  35.,  41.],
       [ 90.,  66.,  37.,  11.,  88.,  41.,  92.,  39.,  60.,  88.],
       [  4.,  43.,  70.,  67.,   8.,  74.,  51.,  58.,  47.,  59.],
       [ 71.,   2.,  56.,  37.,  59.,  59.,  35.,  50.,  17.,  98.],
       [ 83.,  95.,   0.,  82.,  41.,  42.,  52.,  35.,  53.,  56.],
       [ 14.,  15.,  77.,  28.,  28.,  64.,  18.,  42.,  67.,  69.],
       [ 12.,  45.,  22.,  30.,  52.,  24.,  27.,  15.,  20.,  37.],
       [ 73.,  74.,  68.,  32.,  45.,  84.,  13.,  53.,  17.,  32.],
       [ 75.,  14.,  61.,  53.,   

In [5]:
dummy_ts.get(1).sequence()

array([[ 1., 82., 10., 56., 40., 63., 85., 50.,  1., 67.],
       [ 3., 19., 32.,  9., 44., 54., 97., 70., 68., 68.],
       [84., 76., 15., 56., 58., 22., 42., 41., 65., 55.]])

In [6]:
split = (0.7, 0.2, 0.1)
# confirm that split adds up to 1
# assert sum(split) == 1.0, "Split proportions must sum to 1."

(part1, part2, part3) = dummy_ts.split(SplittingStrategy.Temporal, *split)

print(len(part1), len(part2), len(part3))

70 20 10


### Dummy classification dataset

In [8]:
labels = np.ones((100, 1), dtype=np.float64)
labels = labels.astype(str)
labels_list = labels.flatten().tolist()

dummy_class = BaseDataSet.new_classification(
    dummy, labels_list
)

dummy_class.len()

100

### Forecasting Dataset

In [9]:
# Load the dataset
file_path = "LD2011_2014.txt"
df = pd.read_csv(file_path, sep=";", decimal=",")
# drop first column (date)
df = df.drop(columns=["date"])
# turn pandas DataFrame into numPy array
data = df.to_numpy(dtype=np.float64)

data

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.53807107e+00, 2.06258890e+01, 1.73761946e+00, ...,
        1.53589316e+02, 6.70087977e+02, 6.86486486e+03],
       [1.26903553e+00, 2.13371266e+01, 1.73761946e+00, ...,
        1.46911519e+02, 6.46627566e+02, 6.54054054e+03],
       [2.53807107e+00, 1.99146515e+01, 1.73761946e+00, ...,
        1.31886477e+02, 6.73020528e+02, 7.13513514e+03]])

In [10]:
ts = BaseDataSet(data, 3, 3, 3)
ts.set_to_100()

print(type(data))
print(data.shape)

<class 'numpy.ndarray'>
(140256, 370)


In [11]:
(part1, part2, part3) = ts.split(SplittingStrategy.Temporal, 0.7, 0.2, 0.1)

len(part1), len(part2), len(part3)

(98179, 28051, 14026)

### Classification Dataset

In [12]:
from aeon.datasets import load_classification
X, y = load_classification("GunPoint")
X = X.reshape(X.shape[0], -1)

print(type(X))
print(X.shape)

<class 'numpy.ndarray'>
(200, 150)


In [14]:
ts = BaseDataSet.new_classification(X, y)

Usage of RustDataModule


In [17]:
from wrapper import RustDataModule

In [18]:
rust_dm = RustDataModule(data, DatasetType.Forecasting, 3, 3, 3)

In [19]:
rust_dm.setup()

In [20]:
train_dl = rust_dm.train_dataloader()

train_dl

In [21]:
# test wether the dataloader works
for batch in train_dl:
    print(batch)
    print(batch.shape)
    break

tensor([[100.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.]], dtype=torch.float64)
torch.Size([32, 370])
